In [1]:
# tensorflow libraries
import tensorflow as tf
import collections

In [ ]:
def silence_remover(sig, fs, threshold):
    frame_length = int(0.02 * fs)
    nb_frames = len(sig) / frame_length
    average_nrj = np.sum(sig**2)/nb_frames
    print(average_nrj)
    cut_cursor = 0
    while np.sum(sig[cut_cursor:cut_cursor+frame_length]**2) < threshold*average_nrj:
        cut_cursor += frame_length
    return sig[cut_cursor:]

In [ ]:
labels = pd.Series(total_label)
OneH_labels = pd.get_dummies(labels).values

In [ ]:
X_test_shf, X_train_shf, y_test_shf, y_train_shf = get_train_test_set(New_data, OneH_labels, perc_for_test=0.2)
X_train_shf.shape

In [ ]:
# Construct tensorflow graph
d = X_train_shf.shape[1]  # data dimensionality
nc = 15                 # number of classes
Learning_rate = 0.1
tf.reset_default_graph()
# CG inputs
xin = tf.placeholder(tf.float32,[None,d]); print('xin=',xin.get_shape()) # Set the batch size to None such that any batch size can be used in the 
y_label = tf.placeholder(tf.int32,[None, nc]); # neural net
#y_oh = tf.one_hot(y_label, nc, 1.0, 0.0); print('y_oh=',y_oh,y_oh.get_shape())

# Fully Connected layer 1
with tf.name_scope('FC1'):
    W = tf.Variable(tf.truncated_normal([d,2*nc], stddev=0.1 )); #print('W=',W.get_shape())
    b = tf.Variable(tf.zeros([2*nc])); #print('b=',b.get_shape())
    y1 = tf.matmul(xin, W) + b; #print('y1=',y,y.get_shape())
    
    tf.summary.histogram("layer1/weights", W)
    tf.summary.histogram("layer1/biases", b)

# ReLu activation
y2 = tf.nn.relu(y1)
tf.summary.histogram("relu_activation", y2)

# Fully Connected layer 2
with tf.name_scope('FC2'):
    W2 = tf.Variable(tf.truncated_normal([2*nc,nc], stddev=0.1 )); #print('W=',W.get_shape())
    b2 = tf.Variable(tf.zeros([nc]));
    y = tf.matmul(y2, W2) + b2; 

    tf.summary.histogram("layer2/weights", W2)
    tf.summary.histogram("layer2/biases", b2)

# Softmax
y = tf.nn.softmax(y); print('y3=',y,y.get_shape())
tf.summary.histogram("softmax_activation", y)

with tf.name_scope('Loss'):
    # L2 Regularization
    reg_loss = 0.0
    reg_loss += tf.nn.l2_loss(W)
    reg_loss += tf.nn.l2_loss(b)
    reg_loss += tf.nn.l2_loss(W2)
    reg_loss += tf.nn.l2_loss(b2)
    reg_par = 1*1e-3
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y))
    # Loss
    total_loss = cross_entropy #+ reg_par* reg_loss
    tf.summary.scalar("total_loss", total_loss)
    
# Optimization scheme
train_step = tf.train.AdamOptimizer(Learning_rate).minimize(total_loss)

y_out = y
# Accuracy
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

summ = tf.summary.merge_all()

In [ ]:
# Run Graph
n = X_train_shf.shape[0]
indices = collections.deque()
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
test_writer = tf.summary.FileWriter("tensorboard/mlp/test")
train_writer = tf.summary.FileWriter("tensorboard/mlp/train")
train_writer.add_graph(sess.graph)
sess.run(init)
batch_size = 100
for i in range(10001):
    
    # Batch extraction
    if len(indices) < batch_size:
        indices.extend(np.random.permutation(n)) 
    idx = [indices.popleft() for i in range(batch_size)]
    batch_x, batch_y = X_train_shf[idx], y_train_shf[idx]
    #print(batch_x.shape,batch_y.shape)
    
    # save accuracies
    if not i%1000:
        yo, loss, summary_tr, acc_train = sess.run([y_out,total_loss, summ, accuracy], feed_dict={xin: batch_x, y_label: batch_y})
        train_writer.add_summary(summary_tr, i)
        print('\nIteration i=',i,', train accuracy=',acc_train,', loss=',loss)
        #print('\nIteration i=',i,', train accuracy=',acc_train)
        summary_t, acc_test = sess.run([summ, accuracy], feed_dict={xin: X_test_shf, y_label: y_test_shf})
        test_writer.add_summary(summary_t, i)
        print('test accuracy=',acc_test)
    
    else: # train mlp    
        train_accuracy = sess.run([accuracy], feed_dict={xin: batch_x, y_label: batch_y})

In [ ]:
# Split into traing and testing set

X_test_shf, X_train_shf, y_test_shf, y_train_shf = get_train_test_set(X_c, labels, perc_for_test=0.2)
print('train size : ' + str(X_train_shf.shape))
print('test size : ' + str(X_test_shf.shape))
X_train_shf = X_train_shf.astype(float)
X_train_shf = X_train_shf.astype(float)
y_train_shf = y_train_shf.astype(np.int32)

In [ ]:
# Define computational graph (CG)
d = X_train_shf.shape[1]  # number of frames
nb_ch = 13 # number of channels
nc = 15                  # number of classes
Learning_rate = 0.001
batch_size = X_test_shf.shape[0]

# tensor inputs
tf.reset_default_graph()
xin = tf.placeholder(tf.float32,[batch_size,d,nb_ch]); print('xin=',xin,xin.get_shape()) # allow any size of datapoints
y_label = tf.placeholder(tf.int32,[batch_size]); #print('y_label=',y_label,y_label.get_shape())
y_oh = tf.one_hot(y_label, nc); #print('y_oh =',y_oh,y_oh.get_shape())


# Convolutional layer
F_size = 50   # size of the filter
nb_F = 30  # number of filters
ncl = K*K*F
with tf.name_scope('layer1'):
    Wcl = tf.Variable(tf.truncated_normal([F_size,nb_ch,nb_F], stddev=tf.sqrt(2./tf.to_float(ncl)) )); #print('Wcl=',Wcl.get_shape())
    bcl = tf.Variable(tf.zeros([F])); #print('bcl=',bcl.get_shape())
    x = tf.nn.conv1d(xin, Wcl, stride=1 , padding='SAME')
    x += bcl; print('x2=',x.get_shape())

    tf.summary.histogram("layer1/weights", Wcl)
    tf.summary.histogram("layer1/biases", bcl)

# batch normalization
#x = tf.contrib.layers.batch_norm(x)

# dropout
#keep_prob = tf.placeholder(tf.float32)
#tf.summary.scalar('dropout_keep_probability', keep_prob)
#x = tf.nn.dropout(x, keep_prob)

# ReLU activation
with tf.name_scope('Relu_activation'):
    x = tf.nn.relu(x)
    tf.summary.histogram("relu_activation", x)

# Fully Connected layer
nfc = d*nb_F
with tf.name_scope('FC'):
    x = tf.reshape(x, [batch_size,-1]); print('x3=',x.get_shape())
    Wfc = tf.Variable(tf.truncated_normal([nfc,nc], stddev=tf.sqrt(6./tf.to_float(nfc+nc)) )); print('Wfc=',Wfc.get_shape())
    bfc = tf.Variable(tf.zeros([nc])); print('bfc=',bfc.get_shape())
    y = tf.matmul(x, Wfc); print('y1=',y,y.get_shape())
    y += bfc; print('y2=',y,y.get_shape())

# Softmax
with tf.name_scope('softmax_activation'):
    y = tf.nn.softmax(y); print('y3=',y,y.get_shape())

# Loss
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_oh * tf.log(y), 1))
    total_loss = cross_entropy
    tf.summary.scalar("total_loss", total_loss)
print('cross_entropy=',cross_entropy,cross_entropy.get_shape())

# Optimization scheme
#train_step = tf.train.GradientDescentOptimizer(0.025).minimize(total_loss)
train_step = tf.train.AdamOptimizer(Learning_rate).minimize(total_loss)

# Accuracy
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_oh,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

summ = tf.summary.merge_all()

In [ ]:
# Run Computational Graph
n = X_train_shf.shape[0]
indices = collections.deque()
init = tf.global_variables_initializer()
sess = tf.Session()

train_writer = tf.summary.FileWriter('tensorboard/cnn/train',sess.graph)
test_writer = tf.summary.FileWriter('tensorboard/cnn/test')

sess.run(init)
for i in range(10001):
    
    # Batch extraction
    if len(indices) < batch_size:
        indices.extend(np.random.permutation(n)) 
    idx = [indices.popleft() for i in range(batch_size)]
    batch_x, batch_y = X_train_shf[idx], y_train_shf[idx]
    #print(batch_x.shape,batch_y.shape)
    
    # Run CG for variable training
    summary, _, acc_train, total_loss_o = sess.run([summ, train_step,accuracy,total_loss], feed_dict={xin: batch_x, y_label: batch_y})
    train_writer.add_summary(summary, i)
    # Run CG for test set
    if not i%1000:
        print('\nIteration i=',i,', train accuracy=',acc_train,', loss=',total_loss_o)
        acc_test = sess.run(accuracy, feed_dict={xin: X_test_shf, y_label: y_test_shf})
        print('test accuracy=',acc_test)
    if i % 10 == 0:  # Record summaries and test-set accuracy
        acc = sess.run([accuracy], feed_dict={xin: X_test_shf, y_label: y_test_shf})
        test_writer.add_summary(summary_t, i)
        #print('Accuracy at step %s: %s' % (i, acc))

In [ ]:
import scipy

# 3D visualization
import pylab
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot

Cgt = labels
n = dist_matrix.shape[0]

W = scipy.sparse.csc.csc_matrix(dist_matrix)
d = W.sum(axis=0)
d += np.spacing(np.array(0, W.dtype)) # d += epsilon
d = 1.0 / np.sqrt(d)
D = scipy.sparse.diags(d.A.squeeze(), 0)
I = scipy.sparse.identity(d.size, dtype=W.dtype)
L = I - D * W * D

# Compute K smallest eigenvectors/eigenvalues of A
lamb, U = scipy.sparse.linalg.eigsh(L, k=4, which='SM')

# Sort eigenvalue from smallest to largest values
idx = lamb.argsort() # increasing order
lamb, U = lamb[idx], U[:,idx]
print(lamb)

# Y*
Y = U

# Plot in 3D the 2nd, 3rd, 4th columns of Y*
Xdisp = Y[:,1]
Ydisp = Y[:,2]
Zdisp = Y[:,3]

# 2D Visualization
plt.figure(14)
size_vertex_plot = 10
plt.scatter(Xdisp, Ydisp, s=size_vertex_plot*np.ones(n), c=Cgt)
plt.title('2D Visualization')
plt.show()

# 3D Visualization
fig = pylab.figure(15)
ax = Axes3D(fig)
ax.scatter(Xdisp, Ydisp, Zdisp, c=Cgt)
pylab.title('3D Visualization')
pyplot.show()

In [ ]:
plt.specgram(sig, NFFT=512, Fs=fs)
plt.show()